In [ ]:
import os
import tarfile
from six.moves import urllib
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from zlib import crc32
from sklearn.model_selection import StratifiedShuffleSplit

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets","housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# Acquire housing data
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path,"housing.tgz")
    urllib.request.urlretrieve(housing_url,tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

# Load housing data into Pandas Dataframe
def load_housing_data(housing_path=HOUSING_PATH):
    fetch_housing_data()
    csv_path=os.path.join(housing_path,"housing.csv")
    return pd.read_csv(csv_path)

# Ensure that records from the test set are not added back into the train set via randomization
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

# Split data into testing and training portions (20% testing, 80% training)
def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

# Create Pandas dataframe
housing = load_housing_data()

# housing.head() # First 5 records shown
# housing.info() # Info function
# housing.describe() # describe function

# Histogram for data
housing.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
housing_with_id = housing.reset_index()   # adds an `index` column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index") # Creates train/test sets
print(len(train_set)) # Print length of train
print(len(test_set)) # Print length of test

In [ ]:
# The pd.cut() function is used to create an income category attribute with 5 categories (labeled from 1 to 5): category 1 ranges from # 0 to 1.5 (i.e., less than $15,000), category 2 from 1.5 to 3, and so on
housing["income_cat"] = pd.cut(housing["median_income"], bins=[0.,1.5,3.0,4.5,6.,np.inf], labels=[1,2,3,4,5])
housing["income_cat"].hist() # Histogram of income brackets

In [ ]:
# Stratified sampling for income category (making sure the sample represents the population)
split = StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index, test_index in split.split(housing,housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

# Print out percentage of people in each income bracket after stratified sampling
print(strat_test_set["income_cat"].value_counts() / len(strat_test_set))

# Remove "income_cat" to bring data back to original state
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
# Create copy set of training set
housing = strat_train_set.copy()
# Plot housing by population and long/lat
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
# Graph based on population, lat/long, and pricing
housing.plot(kind="scatter",x="longitude",y="latitude",alpha=0.4,s=housing["population"]/100,label="population",figsize=(10,7),c="median_house_value",cmap=plt.get_cmap("jet"),colorbar=True)
plt.legend()

In [ ]:
# Computing the standard correlation coeficient (Pearson's r)
corr_matrix = housing.corr()

# Output how different datapoints correlate to the median house value
corr_matrix['median_house_value'].sort_values(ascending=False)

In [ ]:
# Pandas scatter_matrix function to plot attributes against each other
from pandas.plotting import scatter_matrix
attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))

In [ ]:
# Median income to median housing price
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

In [ ]:
# Creating new attributes
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

# Another look at correlation matrix
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# Reverting back to a clean training set
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

# Dealing with the total_bedrooms attribute missing cells (different options available in notes)
from sklearn.impute import SimpleImputer

# Use SimpleImputer function to fill missing values
imputer = SimpleImputer(strategy="median")
# Drop non-numerical data for SimpleImputer sklearn function
housing_num = housing.drop("ocean_proximity", axis=1)
# Fit imputer to housing_num to learn median values
imputer.fit(housing_num)
# print(imputer.statistics_)
# print(housing_num.median().values)
# Fill x with housing_num + median values for previously empty cells
x = imputer.transform(housing_num) # This results in numpy array
# Transforming back into Pandas dataframe
housing_tr = pd.DataFrame(x, columns=housing_num.columns)

# Working with the ocean_proximity attribute
housing_cat = housing[["ocean_proximity"]]
# housing_cat.head(10)

# Import OrdinalEncoder to transform ocean_proximity into numerical data
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
# Note: We are using OneHotEncoder, not OrdinalEncoder. Reasoning in notes under 'transforming into numerical data'
# However, it is still encoded/transformed in both ways in this program for educational purposes

# Create OrdinalEncoder object
ordinal_encoder = OrdinalEncoder()
# Encode/transform dataset
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
# print(housing_cat_encoded[:10])
# print(ordinal_encoder.categories_)

# Encoder object created
cat_encoder = OneHotEncoder()
# Data encoded/transformed
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
# print(housing_cat_1hot)
# housing_cat_1hot is currently a SciPy sparse matrix. It is useful in this format, however if someone wanted to use a 2d array
# instead they can use the toarray() method


# To implement transformer class reference 'sklearn-transformer-class.py' in the hands_on_ml folder
from sklearn.base import BaseEstimator, TransformerMixin

# get the right column indices: safer than hard-coding indices 3, 4, 5, 6
rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kwargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()


# Sklearn Pipeline class for data transformation
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
from sklearn.compose import ColumnTransformer

# Explanation of code below
'''
Here is how this works: first we import the ColumnTransformer class, next we get the list of numerical column names and the list of categorical column names, and we construct a ColumnTransformer. The constructor requires a list of tuples, where each tuple contains a name22, a transformer and a list of names (or indices) of columns that the transformer should be applied to. In this example, we specify that the numerical columns should be transformed using the num_pipeline that we defined earlier, and the categorical columns should be transformed using a OneHotEncoder. Finally, we apply this ColumnTransformer to the housing data: it applies each transformer to the appropriate columns and concatenates the outputs along the second axis (the transformers must return the same number of rows). 
'''

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
("cat", OneHotEncoder(), cat_attribs)
])

housing_prepared = full_pipeline.fit_transform(housing)

# Data cleaning is finished

In [ ]:
# Create ML model
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Testing model on a few instances of the training set
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions: ", lin_reg.predict(some_data_prepared))
print("Labels: ", list(some_labels))

In [ ]:
# Measure the regression models RMSE on the entire training set
# Notes on RMSE in hands_on_ml_notes.py
from sklearn.metrics import mean_squared_error

# 
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

In [ ]:
# Implement more complex model to increase precision, decrease loss
# Initially this will cause significant overfitting
from sklearn.tree import DecisionTreeRegressor

# New model
tree_reg = DecisionTreeRegressor()
# Fit
tree_reg.fit(housing_prepared, housing_labels)


In [ ]:
# Accuracy of new model - New model causing overfit of data
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels,housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

In [ ]:
# The next step is to remove overfitting

# Better evaluation may be achieved using cross-validation
# e.g split training set into a smaller training set and a validation set
# this can be done using the train_test_split function
# One alternative to this is to use sklearns k-fold-cross-validation feature

from sklearn.model_selection import cross_val_score

# Split training set into 10 distinct subsets called folds, then train and evaluate decision tree model 10 times
# the result is an array containing the 10 evaluation scores
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)



In [ ]:
# Print results of cross_val_score operation
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
# Calculate scores for linear regression model
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)

# Display scores, mean, standard deviation
display_scores(lin_rmse_scores)

# Results discover decision tree model is overfitting the data so significantly that the linear regression model is working
# more effectively

In [ ]:
# Next model -- RandomForestRegressor (Chp. 7)
# Information
'''
Random Forests work by training many decision trees on random subsets of the features, then averaging out their predictions.
Building a model on top of many other models is called ensemble learning.
'''

from sklearn.ensemble import RandomForestRegressor

# Create model
forest_reg = RandomForestRegressor()
# Fit model
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Calculate RMSE
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)

# Print Score,mean,etc
display_scores(forest_rmse_scores)

In [ ]:
# Code for saving models to re-implement later
'''
from sklearn.externals import joblib

joblib.dump(my_model, "my_model.pkl") # Saving the model
my_model_loaded = joblib.load("my_model.pkl") # Re-loading the model
'''

# Fine tuning the model (Hyperparameters, etc.)

# Methods
'''
1. Grid search
    Scikit-Learn's GridSearchCV is used for grid searching.
    Input: Which hyperparameters to use. What values to try out.

    Example code available below
'''

from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

# Code is working on forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)